In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [5]:
encounters = pd.read_csv("data/encounters.csv")
organizations = pd.read_csv("data/organizations.csv")
patients = pd.read_csv("data/patients.csv")
payers = pd.read_csv("data/payers.csv")
procedures = pd.read_csv("data/procedures.csv")

In [6]:
encounters['START'] = pd.to_datetime(encounters['START'])
encounters['STOP'] = pd.to_datetime(encounters['STOP'])
encounters['START_MONTH'] = encounters['START'].dt.strftime('%Y-%m')
encounters['STAY_DURATION'] = (encounters['STOP'] - encounters['START']) / pd.Timedelta(hours=1)

In [7]:
current_date = encounters['START'].max().date()
current_date

datetime.date(2022, 2, 5)

In [8]:
age_groups = {
    '0 – 14 years': [0, 14],
    '15 – 24 years': [15, 24],
    '25 – 54 years': [25, 54],
    '55 – 64 years': [55, 64],
    '65 years and over': [65, 999]
}

In [9]:
df_admissions = pd.merge(
    encounters[['Id', 'START', 'START_MONTH', 'PATIENT', 'ENCOUNTERCLASS']],
    patients[['Id', 'BIRTHDATE', 'GENDER']].rename(columns={'Id': 'PATIENT'}),
    on='PATIENT',
    how='left'
)

df_admissions['AGE'] = (pd.to_datetime(current_date) - pd.to_datetime(df_admissions['BIRTHDATE'])).apply(lambda x: np.floor(x.days/365.25)).astype(int)
df_admissions['AGE_GROUP'] = None
for age_group, age_range in age_groups.items():
    df_admissions['AGE_GROUP'] = np.where(
        (df_admissions['AGE'] >= age_range[0]) & (df_admissions['AGE'] <= age_range[1]),
        age_group,
        df_admissions['AGE_GROUP']
    )

df_admissions['IS_ADMISSION'] = np.where(df_admissions['ENCOUNTERCLASS'] == 'inpatient', 1, 0)
df_admissions['ADMISSION_ORDER'] = df_admissions.groupby(['PATIENT', 'IS_ADMISSION'])['START'].rank().astype(int)
df_admissions['IS_READMISSION'] = np.where((df_admissions['IS_ADMISSION'] == 1) & (df_admissions['ADMISSION_ORDER'] > 1), 1, 0)

df_admissions

,Id,START,START_MONTH,PATIENT,ENCOUNTERCLASS,BIRTHDATE,GENDER,AGE,AGE_GROUP,IS_ADMISSION,ADMISSION_ORDER,IS_READMISSION
0,32c84703-2481-49cd-d571-3899d5820253,2011-01-02 09:26:36+00:00,2011-01,3de74169-7f67-9304-91d4-757e0f3a14d2,ambulatory,1928-08-25,M,93,65 years and over,0,1,0
1,c98059da-320a-c0a6-fced-c8815f3e3f39,2011-01-03 05:44:39+00:00,2011-01,d9ec2e44-32e9-9148-179a-1653348cc4e2,outpatient,1964-01-05,F,58,55 – 64 years,0,1,0
2,4ad28a3a-2479-782b-f29c-d5b3f41a001e,2011-01-03 14:32:11+00:00,2011-01,73babadf-5b2b-fee7-189e-6f41ff213e01,outpatient,1924-06-30,F,97,65 years and over,0,1,0
3,c3f4da61-e4b4-21d5-587a-fbc89943bc19,2011-01-03 16:24:45+00:00,2011-01,3b46a0b7-0f34-9b9a-c319-ace4a1f58c0b,wellness,1923-05-21,M,98,65 years and over,0,1,0
4,a9183b4f-2572-72ea-54c2-b3cd038b4be7,2011-01-03 17:36:53+00:00,2011-01,fa006887-d93c-d302-8b89-f3c25f88c0e1,ambulatory,1952-11-02,M,69,65 years and over,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
27886,917bb534-2ed3-e1f5-ac27-74f314d19c63,2022-01-29 11:42:06+00:00,2022-01,0c122290-8c15-77ac-b418-2fea4e8a8e6a,outpatient,1974-04-27,M,47,25 – 54 years,0,27,0
27887,07710480-9d6b-9c9b-87c3-c1d54df4069d,2022-01-29 20:12:53+00:00,2022-01,20a4bc24-6b69-2f5c-dc74-1df390cae25b,urgentcare,1933-12-16,M,88,65 years and over,0,14,0
27888,01b57f06-cebe-a3e4-4423-a796ffb0c35d,2022-01-29 20:35:37+00:00,2022-01,ff1b3c26-53a6-4590-ce79-a3f7269274ea,ambulatory,1990-09-22,F,31,25 – 54 years,0,71,0
27889,7d435668-0813-eb3b-0f26-cb741fb39561,2022-01-29 20:35:37+00:00,2022-01,ff1b3c26-53a6-4590-ce79-a3f7269274ea,wellness,1990-09-22,F,31,25 – 54 years,0,71,0


In [10]:
df_admissions_grouped = df_admissions.groupby('START_MONTH').agg(
    PATIENTS = ('PATIENT', 'nunique'),
    ADMISSIONS = ('IS_ADMISSION', 'sum'),
    READMISSIONS = ('IS_READMISSION', 'sum'), 
)

df_admissions_grouped

,PATIENTS,ADMISSIONS,READMISSIONS
START_MONTH,,,
2011-01,68,7,0
2011-02,67,8,5
2011-03,85,6,6
2011-04,72,5,5
2011-05,78,7,7
...,...,...,...
2021-10,119,7,5
2021-11,134,4,4
2021-12,110,2,1


In [11]:
df_readmissions_grouped = df_admissions.groupby(['START_MONTH', 'AGE_GROUP']).agg(
    PATIENTS = ('PATIENT', 'nunique'),
    ADMISSIONS = ('IS_ADMISSION', 'sum'),
    READMISSIONS = ('IS_READMISSION', 'sum'),
)

df_readmissions_grouped

PATIENTS  ADMISSIONS  READMISSIONS
START_MONTH AGE_GROUP                                            
2011-01     25 – 54 years            10           1             0
            55 – 64 years             5           0             0
            65 years and over        53           6             0
2011-02     25 – 54 years             7           0             0
            65 years and over        60           8             5
...                                 ...         ...           ...
2021-12     65 years and over        76           2             1
2022-01     25 – 54 years            25           1             1
            55 – 64 years            12           0             0
            65 years and over        66           3             3
2022-02     65 years and over         1           0             0

[399 rows x 3 columns]

In [12]:
df_length = pd.merge(
    encounters[['Id', 'START', 'STOP', 'START_MONTH', 'PATIENT', 'STAY_DURATION']],
    patients[['Id', 'BIRTHDATE', 'GENDER']].rename(columns={'Id': 'PATIENT'}),
    on='PATIENT',
    how='left'
)

df_length['AGE'] = (pd.to_datetime(current_date) - pd.to_datetime(df_length['BIRTHDATE'])).apply(lambda x: np.floor(x.days/365.25)).astype(int)
df_length['AGE_GROUP'] = None
for age_group, age_range in age_groups.items():
    df_length['AGE_GROUP'] = np.where(
        (df_length['AGE'] >= age_range[0]) & (df_length['AGE'] <= age_range[1]),
        age_group,
        df_length['AGE_GROUP']
    )

df_length

,Id,START,STOP,START_MONTH,PATIENT,STAY_DURATION,BIRTHDATE,GENDER,AGE,AGE_GROUP
0,32c84703-2481-49cd-d571-3899d5820253,2011-01-02 09:26:36+00:00,2011-01-02 12:58:36+00:00,2011-01,3de74169-7f67-9304-91d4-757e0f3a14d2,3.533333,1928-08-25,M,93,65 years and over
1,c98059da-320a-c0a6-fced-c8815f3e3f39,2011-01-03 05:44:39+00:00,2011-01-03 06:01:42+00:00,2011-01,d9ec2e44-32e9-9148-179a-1653348cc4e2,0.284167,1964-01-05,F,58,55 – 64 years
2,4ad28a3a-2479-782b-f29c-d5b3f41a001e,2011-01-03 14:32:11+00:00,2011-01-03 14:47:11+00:00,2011-01,73babadf-5b2b-fee7-189e-6f41ff213e01,0.250000,1924-06-30,F,97,65 years and over
3,c3f4da61-e4b4-21d5-587a-fbc89943bc19,2011-01-03 16:24:45+00:00,2011-01-03 16:39:45+00:00,2011-01,3b46a0b7-0f34-9b9a-c319-ace4a1f58c0b,0.250000,1923-05-21,M,98,65 years and over
4,a9183b4f-2572-72ea-54c2-b3cd038b4be7,2011-01-03 17:36:53+00:00,2011-01-03 17:51:53+00:00,2011-01,fa006887-d93c-d302-8b89-f3c25f88c0e1,0.250000,1952-11-02,M,69,65 years and over
...,...,...,...,...,...,...,...,...,...,...
27886,917bb534-2ed3-e1f5-ac27-74f314d19c63,2022-01-29 11:42:06+00:00,2022-01-29 11:57:06+00:00,2022-01,0c122290-8c15-77ac-b418-2fea4e8a8e6a,0.250000,1974-04-27,M,47,25 – 54 years
27887,07710480-9d6b-9c9b-87c3-c1d54df4069d,2022-01-29 20:12:53+00:00,2022-01-29 20:27:53+00:00,2022-01,20a4bc24-6b69-2f5c-dc74-1df390cae25b,0.250000,1933-12-16,M,88,65 years and over
27888,01b57f06-cebe-a3e4-4423-a796ffb0c35d,2022-01-29 20:35:37+00:00,2022-01-29 20:50:37+00:00,2022-01,ff1b3c26-53a6-4590-ce79-a3f7269274ea,0.250000,1990-09-22,F,31,25 – 54 years
27889,7d435668-0813-eb3b-0f26-cb741fb39561,2022-01-29 20:35:37+00:00,2022-01-29 20:50:37+00:00,2022-01,ff1b3c26-53a6-4590-ce79-a3f7269274ea,0.250000,1990-09-22,F,31,25 – 54 years


In [13]:
df_length_grouped = df_length.groupby('START_MONTH').agg(
    AVERAGE_DURATION = ('STAY_DURATION', 'mean')
)

df_length_grouped

,AVERAGE_DURATION
START_MONTH,
2011-01,2.651618
2011-02,2.816424
2011-03,1.641191
2011-04,1.578609
2011-05,1.757937
...,...
2021-10,1.500546
2021-11,1.076332
2021-12,0.973694


In [14]:
df_length_grouped = df_length.groupby(['START_MONTH', 'AGE_GROUP']).agg(
    AVERAGE_DURATION = ('STAY_DURATION', 'mean')
)

df_length_grouped

AVERAGE_DURATION
START_MONTH AGE_GROUP                          
2011-01     25 – 54 years              2.625222
            55 – 64 years              0.256833
            65 years and over          2.823983
2011-02     25 – 54 years              0.250000
            65 years and over          3.043829
...                                         ...
2021-12     65 years and over          1.116704
2022-01     25 – 54 years              1.050118
            55 – 64 years              0.250000
            65 years and over          1.254304
2022-02     65 years and over          0.250000

[399 rows x 1 columns]

In [15]:
df_cost = encounters[['Id', 'START', 'START_MONTH', 'ENCOUNTERCLASS', 'TOTAL_CLAIM_COST']]

In [16]:
df_cost.groupby('START_MONTH').agg(
    AVERAGE_COST = ('TOTAL_CLAIM_COST', 'mean')
)

,AVERAGE_COST
START_MONTH,
2011-01,3013.043488
2011-02,3555.830000
2011-03,2536.631860
2011-04,3420.855221
2011-05,3187.153893
...,...
2021-10,4575.415244
2021-11,3104.703296
2021-12,3448.615758


In [17]:
df_cost.groupby(['START_MONTH', 'ENCOUNTERCLASS']).agg(
    AVERAGE_COST = ('TOTAL_CLAIM_COST', 'mean')
)

AVERAGE_COST
START_MONTH ENCOUNTERCLASS              
2011-01     ambulatory       2543.549677
            emergency         586.692500
            inpatient       10305.511429
            outpatient       1771.302581
            urgentcare        278.580000
...                                  ...
2022-01     inpatient         743.057500
            outpatient       3810.240811
            urgentcare       5684.328571
            wellness         4687.473636
2022-02     wellness            0.000000

[797 rows x 1 columns]

In [18]:
df_encounter_coverage = pd.merge(
    encounters[['Id', 'PATIENT', 'START', 'START_MONTH', 'TOTAL_CLAIM_COST', 'PAYER_COVERAGE', 'PAYER']],
    payers[['Id', 'NAME']].rename(columns={'Id': 'PAYER'}),
    on='PAYER',
    how='left'
)

df_encounter_coverage['IS_COVERED'] = np.where(df_encounter_coverage['NAME'] == 'NO_INSURANCE', 0, 1)

df_procedures = procedures.groupby(['ENCOUNTER', 'PATIENT']).agg(
    PROCEDURES = ('ENCOUNTER', 'count'),
    TOTAL_PROCEDURE_COST = ('BASE_COST', 'sum')
).reset_index()

df_encounter_coverage = df_encounter_coverage.merge(
    df_procedures.rename(columns={'ENCOUNTER': 'Id'}),
    on=['Id', 'PATIENT'],
    how='left'
)

df_encounter_coverage['PROCEDURES'] = df_encounter_coverage['PROCEDURES'].fillna(0).astype(int)
df_encounter_coverage['TOTAL_PROCEDURE_COST'] = df_encounter_coverage['TOTAL_PROCEDURE_COST'].fillna(0)

df_encounter_coverage

,Id,PATIENT,START,START_MONTH,TOTAL_CLAIM_COST,PAYER_COVERAGE,PAYER,NAME,IS_COVERED,PROCEDURES,TOTAL_PROCEDURE_COST
0,32c84703-2481-49cd-d571-3899d5820253,3de74169-7f67-9304-91d4-757e0f3a14d2,2011-01-02 09:26:36+00:00,2011-01,1018.02,0.00,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,NO_INSURANCE,0,1,903.0
1,c98059da-320a-c0a6-fced-c8815f3e3f39,d9ec2e44-32e9-9148-179a-1653348cc4e2,2011-01-03 05:44:39+00:00,2011-01,2619.36,0.00,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,NO_INSURANCE,0,1,2477.0
2,4ad28a3a-2479-782b-f29c-d5b3f41a001e,73babadf-5b2b-fee7-189e-6f41ff213e01,2011-01-03 14:32:11+00:00,2011-01,461.59,305.27,7caa7254-5050-3b5e-9eae-bd5ea30e809c,Medicare,1,0,0.0
3,c3f4da61-e4b4-21d5-587a-fbc89943bc19,3b46a0b7-0f34-9b9a-c319-ace4a1f58c0b,2011-01-03 16:24:45+00:00,2011-01,1784.24,0.00,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,NO_INSURANCE,0,0,0.0
4,a9183b4f-2572-72ea-54c2-b3cd038b4be7,fa006887-d93c-d302-8b89-f3c25f88c0e1,2011-01-03 17:36:53+00:00,2011-01,234.72,0.00,42c4fca7-f8a9-3cd1-982a-dd9751bf3e2a,Anthem,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
27886,917bb534-2ed3-e1f5-ac27-74f314d19c63,0c122290-8c15-77ac-b418-2fea4e8a8e6a,2022-01-29 11:42:06+00:00,2022-01,85.55,24.27,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,Medicaid,1,0,0.0
27887,07710480-9d6b-9c9b-87c3-c1d54df4069d,20a4bc24-6b69-2f5c-dc74-1df390cae25b,2022-01-29 20:12:53+00:00,2022-01,10588.34,8438.67,7caa7254-5050-3b5e-9eae-bd5ea30e809c,Medicare,1,1,10446.0
27888,01b57f06-cebe-a3e4-4423-a796ffb0c35d,ff1b3c26-53a6-4590-ce79-a3f7269274ea,2022-01-29 20:35:37+00:00,2022-01,11984.29,0.00,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,NO_INSURANCE,0,5,12686.0
27889,7d435668-0813-eb3b-0f26-cb741fb39561,ff1b3c26-53a6-4590-ce79-a3f7269274ea,2022-01-29 20:35:37+00:00,2022-01,408.80,0.00,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,NO_INSURANCE,0,0,0.0


In [19]:
df_encounter_coverage_grouped = df_encounter_coverage.groupby(['START_MONTH', 'IS_COVERED']).agg(
    PROCEDURES = ('PROCEDURES', 'sum'),
    TOTAL_PROCEDURE_COST = ('TOTAL_PROCEDURE_COST', 'sum')
)

df_encounter_coverage_grouped

PROCEDURES  TOTAL_PROCEDURE_COST
START_MONTH IS_COVERED                                  
2011-01     0                   27               43122.0
            1                   51              168711.0
2011-02     0                   33               70880.0
            1                   92              194606.0
2011-03     0                   26               97430.0
...                            ...                   ...
2021-12     0                   81              512162.0
            1                  235              312540.0
2022-01     0                   99              340579.0
            1                  167              373484.0
2022-02     1                    0                   0.0

[267 rows x 2 columns]

In [38]:
procedures

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,REASONCODE,REASONDESCRIPTION
0,2011-01-02T09:26:36Z,2011-01-02T12:58:36Z,3de74169-7f67-9304-91d4-757e0f3a14d2,32c84703-2481-49cd-d571-3899d5820253,265764009,Renal dialysis (procedure),903,NaN,NaN
1,2011-01-03T05:44:39Z,2011-01-03T06:01:42Z,d9ec2e44-32e9-9148-179a-1653348cc4e2,c98059da-320a-c0a6-fced-c8815f3e3f39,76601001,Intramuscular injection,2477,NaN,NaN
2,2011-01-04T14:49:55Z,2011-01-04T15:04:55Z,d856d6e6-4c98-e7a2-129b-44076c63d008,2cfd4ddd-ad13-fe1e-528b-15051cea2ec3,703423002,Combined chemotherapy and radiation therapy (p...,11620,363406005.0,Malignant tumor of colon
3,2011-01-05T04:02:09Z,2011-01-05T04:17:09Z,bc9d59c3-0a30-6e3b-f47d-022e4f03c8de,17966936-0878-f4db-128b-a43ae10d0878,173160006,Diagnostic fiberoptic bronchoscopy (procedure),9796,162573006.0,Suspected lung cancer (situation)
4,2011-01-05T12:58:36Z,2011-01-05T16:42:36Z,3de74169-7f67-9304-91d4-757e0f3a14d2,9de5f0b0-4ba4-ce6f-45fb-b55c202f31a5,265764009,Renal dialysis (procedure),1255,NaN,NaN
...,...,...,...,...,...,...,...,...,...
47696,2022-01-29T20:35:37Z,2022-01-29T20:50:37Z,ff1b3c26-53a6-4590-ce79-a3f7269274ea,01b57f06-cebe-a3e4-4423-a796ffb0c35d,430193006,Medication Reconciliation (procedure),413,NaN,NaN
47697,2022-01-29T20:35:37Z,2022-01-29T20:50:37Z,ff1b3c26-53a6-4590-ce79-a3f7269274ea,01b57f06-cebe-a3e4-4423-a796ffb0c35d,274804006,Evaluation of uterine fundal height,3238,72892002.0,Normal pregnancy
47698,2022-01-29T20:35:37Z,2022-01-29T20:50:37Z,ff1b3c26-53a6-4590-ce79-a3f7269274ea,01b57f06-cebe-a3e4-4423-a796ffb0c35d,225158009,Auscultation of the fetal heart,7045,72892002.0,Normal pregnancy
47699,2022-01-29T20:35:37Z,2022-01-29T20:50:37Z,ff1b3c26-53a6-4590-ce79-a3f7269274ea,01b57f06-cebe-a3e4-4423-a796ffb0c35d,118001005,Streptococcus pneumoniae group B antigen test,1559,72892002.0,Normal pregnancy


In [44]:
df_procedure_coverage = pd.merge(
    procedures[['ENCOUNTER', 'PATIENT', 'DESCRIPTION', 'BASE_COST']].rename(columns={'ENCOUNTER': 'Id'}),
    df_encounter_coverage[['Id', 'PATIENT', 'START', 'START_MONTH', 'IS_COVERED']],
    on=['Id', 'PATIENT'],
    how='left'
)

df_procedure_coverage

,Id,PATIENT,DESCRIPTION,BASE_COST,START,START_MONTH,IS_COVERED
0,32c84703-2481-49cd-d571-3899d5820253,3de74169-7f67-9304-91d4-757e0f3a14d2,Renal dialysis (procedure),903,2011-01-02 09:26:36+00:00,2011-01,0
1,c98059da-320a-c0a6-fced-c8815f3e3f39,d9ec2e44-32e9-9148-179a-1653348cc4e2,Intramuscular injection,2477,2011-01-03 05:44:39+00:00,2011-01,0
2,2cfd4ddd-ad13-fe1e-528b-15051cea2ec3,d856d6e6-4c98-e7a2-129b-44076c63d008,Combined chemotherapy and radiation therapy (p...,11620,2011-01-04 14:49:55+00:00,2011-01,1
3,17966936-0878-f4db-128b-a43ae10d0878,bc9d59c3-0a30-6e3b-f47d-022e4f03c8de,Diagnostic fiberoptic bronchoscopy (procedure),9796,2011-01-05 04:02:09+00:00,2011-01,1
4,9de5f0b0-4ba4-ce6f-45fb-b55c202f31a5,3de74169-7f67-9304-91d4-757e0f3a14d2,Renal dialysis (procedure),1255,2011-01-05 12:58:36+00:00,2011-01,0
...,...,...,...,...,...,...,...
47696,01b57f06-cebe-a3e4-4423-a796ffb0c35d,ff1b3c26-53a6-4590-ce79-a3f7269274ea,Medication Reconciliation (procedure),413,2022-01-29 20:35:37+00:00,2022-01,0
47697,01b57f06-cebe-a3e4-4423-a796ffb0c35d,ff1b3c26-53a6-4590-ce79-a3f7269274ea,Evaluation of uterine fundal height,3238,2022-01-29 20:35:37+00:00,2022-01,0
47698,01b57f06-cebe-a3e4-4423-a796ffb0c35d,ff1b3c26-53a6-4590-ce79-a3f7269274ea,Auscultation of the fetal heart,7045,2022-01-29 20:35:37+00:00,2022-01,0
47699,01b57f06-cebe-a3e4-4423-a796ffb0c35d,ff1b3c26-53a6-4590-ce79-a3f7269274ea,Streptococcus pneumoniae group B antigen test,1559,2022-01-29 20:35:37+00:00,2022-01,0


In [47]:
df_procedure_coverage_grouped = df_procedure_coverage[df_procedure_coverage['IS_COVERED'] == 0].groupby(['START_MONTH', 'DESCRIPTION']).agg(
    BASE_COST = ('BASE_COST', 'mean')
)

df_procedure_coverage_grouped

BASE_COST
START_MONTH DESCRIPTION                                                  
2011-01     Assessment of anxiety (procedure)                       431.0
            Assessment of health and social care needs (pro...      431.0
            Assessment of substance use (procedure)                 431.0
            Assessment using Morse Fall Scale (procedure)           431.0
            Combined chemotherapy and radiation therapy (pr...    11081.0
...                                                                   ...
2022-01     Screening for domestic abuse (procedure)                431.0
            Screening for drug abuse (procedure)                    431.0
            Streptococcus pneumoniae group B antigen test          1559.0
            Urine screening for glucose                            1619.0
            Vaccination for diphtheria  pertussis  and tetanus     3167.0

[4046 rows x 1 columns]